In [1]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from PIL import Image
import os

In [2]:
plt.rc('font', family='NanumBarunGothic')

In [3]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
# 동화 설정

# 동화 제목
story_title = input('동화 제목 : ')

# 페이지 수 설정
while True:
    try:
        pages = int(input("몇 페이지 ? (숫자만 입력하세요): "))
        if pages > 10:
            print("❌ 10페이지 이하로 해주세요")
            continue
        elif pages < 1:
            print("❌ 1페이지 이상으로 해주세요")
            continue
        print(f"페이지 수는 {pages}장 입니다")
        break
    except ValueError:
        print("❌ 숫자가 아닙니다. 다시 입력해주세요!")

# 인물 수 설정
while True:
    try:
        characters = int(input("인물 수 : "))
        if characters > 5:
            print("❌ 인물은 5명 이하로 해주세요")
            continue
        elif characters < 1:
          print("❌ 인물은 1명 이상으로 해주세요")
          continue
        print(f"인물 수는 {characters}명 입니다")
        break
    except ValueError:
        print("❌ 숫자가 아닙니다. 다시 입력해주세요!")


In [ ]:
# 캐릭터 설정
character_style_dic = {}
character_style_list = []
total_style = "A cute, cartoon-style children's book illustration, featuring soft pastel colors, a warm and friendly atmosphere, playful and charming characters, simple and clear details, suitable for young kids."

for _ in range(characters):
  name = input('인물 이름 : ')
  style = input('인물의 생김새 입력하세요 : ')
  character_style_dic = {'name':name,'style':style}
  character_style_list.append(character_style_dic)

final_characters_style = []
for item in character_style_list:
    updated_item = {
        'name': item['name'],
        'style': f"{item['style']}, {total_style}"
    }
    final_characters_style.append(updated_item)

In [ ]:
character_style_list

In [ ]:
final_characters_style

In [ ]:
def character_sketch(final_characters_style, seed=42):
    """캐릭터를 그리는 함수"""
    # 결과 저장할 폴더 생성
    os.makedirs("storybook_images", exist_ok=True)

    # 일관성을 위해 시드 설정
    generator = torch.Generator(device="cuda").manual_seed(seed)

    generated_characters = []

    for i, character in enumerate(final_characters_style):
        print(f"캐릭터 {character['name']} 생성 중...")

        # 이미지 생성
        image = pipe(
            character['style'],
            generator=generator,
            num_inference_steps=30,
            guidance_scale=7.5
        ).images[0]

        # 이미지 저장
        filename = f"storybook_images/{character['name']}_character.png"
        image.save(filename)

        # 생성된 캐릭터 정보 저장
        generated_characters.append({
            'name': character['name'],
            'style': character['style'],
            'image': image,
            'filename': filename
        })

        print(f"캐릭터 {character['name']} 저장 완료: {filename}")

    return generated_characters

In [ ]:
# 캐릭터 생성
generated_characters = character_sketch(final_characters_style)

# 생성된 캐릭터들 한번에 출력
def display_characters(generated_characters):
    fig, axes = plt.subplots(1, len(generated_characters), figsize=(5*len(generated_characters), 5))

    if len(generated_characters) == 1:
        axes = [axes]  # 단일 이미지일 때 리스트로 변환

    for i, character in enumerate(generated_characters):
        axes[i].imshow(character['image'])
        axes[i].set_title(f"{character['name']}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# 출력 실행
display_characters(generated_characters)

In [ ]:
# 각 페이지별 스토리와 프롬프트

story_pages = []
page_dic = {}
story = ''
prompt = ''

for page in range(1,pages+1):
  story = input('내용을 입력하세요 : ')
  prompt = input('프롬프트를 입력하세요 : ')
  page_dic = {'page':page,'story':story,'prompt':prompt}
  story_pages.append(page_dic)

In [ ]:
story_pages

In [ ]:
# 이미지 생성 및 저장
def generate_storybook_images(story_pages, seed=42):
    """동화책 이미지들을 생성하고 저장하는 함수"""

    # 결과 저장할 폴더 생성
    os.makedirs("storybook_images", exist_ok=True)

    # 일관성을 위해 시드 설정
    generator = torch.Generator(device="cuda").manual_seed(seed)

    generated_images = []

    for page_info in story_pages:
        print(f"페이지 {page_info['page']} 생성 중...")
        print(f"텍스트: {page_info['story']}")

        # 이미지 생성
        image = pipe(
            page_info['prompt'],
            generator=generator,
            num_inference_steps=30,
            guidance_scale=7.5
        ).images[0]

        # 이미지 저장
        filename = f"storybook_images/page_{page_info['page']:02d}.png"
        image.save(filename)

        generated_images.append({
            'page': page_info['page'],
            'image': image,
            'story': page_info['story'],
            'filename': filename
        })

        print(f"저장됨: {filename}\n")

    return generated_images


In [ ]:
# 이미지 생성 실행
images = generate_storybook_images(story_pages)

# 생성된 이미지들을 한 번에 보기
def display_storybook(images):
    """생성된 동화책 이미지들을 예쁘게 출력"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(story_title, fontsize=20, fontweight='bold')

    # 2x3 그리드로 배치 (5개 이미지 + 1개 빈 공간)
    positions = [(0,0), (0,1), (0,2), (1,0), (1,1)]

    for i, img_info in enumerate(images):
        if i < len(positions):
            row, col = positions[i]
            axes[row, col].imshow(img_info['image'])
            axes[row, col].set_title(f"페이지 {img_info['page']}", fontsize=14, fontweight='bold')
            axes[row, col].axis('off')

            # 텍스트를 이미지 아래에 추가
            axes[row, col].story(0.5, -0.15, img_info['story'],
                              transform=axes[row, col].transAxes,
                              ha='center', va='top', fontsize=10,
                              wrap=True, bbox=dict(boxstyle="round,pad=0.3",
                              facecolor="lightblue", alpha=0.7))

    # 마지막 빈 공간 숨기기
    axes[1, 2].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# 동화책 이미지들 출력
display_storybook(images)